In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import requests 
import io
import seaborn as sns
import scipy.sparse as sp

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

import string
from nltk import word_tokenize
from nltk.corpus import stopwords

from bs4 import BeautifulSoup

nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 7.1 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 
     |████████████████████████████████| 3.3 MB 36.4 MB/s 
     |████████████████████████████████| 596 kB 63.9 MB/s 
     |████████████████████████████████| 895 kB 54.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer,BertModel, BertConfig
import torch

### User Mapping

In [ ]:
orig_members = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_members.csv')

In [ ]:
sorted_member_id = orig_members['member_id'].sort_values()
member_id_map ={}
start_id = 1
for id in sorted_member_id:
  member_id_map[id] = start_id
  start_id = start_id + 1

### Notifications

In [ ]:
orig_inline_notifications = pd.read_csv('/content/drive/MyDrive/text gcn/Hackforum data/CSV Data IronMarch/data_csv/orig_inline_notifications.csv')

In [ ]:
orig_inline_notifications.drop(orig_inline_notifications.columns[[0,1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13]], axis =1, inplace=True)

### Functions

In [ ]:
def preprocess_text(text):

    soup = BeautifulSoup(text, "lxml")
    text = soup.body.text

    # Get a list of sentences, removes "\n"
    lst = sent_tokenize(text)
    text = " ".join(lst)

    text = text.replace("\n", "")

    # Preprocess using NLTK
    text = text.lower()

    # Remove Punctuation
    text_p = "".join([char for char in text if char not in string.punctuation])

    # Get a List of words 
    words = word_tokenize(text_p)

    words = " ".join(words)

    # Remove stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]

    ret = " ".join(filtered_words)

    return ret

In [ ]:
def getBERT(post):

  tokens = tokenizer.encode_plus(post,return_tensors='pt')
  
  indexed_tokens = tokens['input_ids']
  tokens_tensor = indexed_tokens.to('cuda')

  with torch.no_grad():
    # See the models docstrings for the detail of the inputs
    outputs = model(tokens_tensor)
    last_hidden_states = outputs.last_hidden_state  

# # Take the average
  n_tensor = last_hidden_states.shape[1]
  vector = (last_hidden_states.sum(axis=1)/n_tensor)

  return vector

In [ ]:
def get_word_split(text1):
  l_total = []
  l_partial = []
  if len(text1.split())//150 >0:
    n = len(text1.split())//150 + 1
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_partial = text1.split()[:200]
      l_total.append(" ".join(l_partial))
    else:
      l_partial = text1.split()[w*150:w*150 + 200]
      l_total.append(" ".join(l_partial))
  return l_total

### Variables

In [ ]:
embedding = np.zeros((orig_members.shape[0], 768))
count = np.zeros(orig_members.shape[0])
embedding = torch.tensor(embedding)
embedding = embedding.to('cuda')
count = torch.tensor(count)
count = count.to('cuda')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

### Iterations

In [ ]:
for i, row in orig_inline_notifications.iterrows():
  row = orig_inline_notifications.iloc[i]
  author_id = row['notify_from_id']
  if author_id not in member_id_map:
          continue
  id = member_id_map[author_id]
  doc = row['notify_text']

  clean_doc = preprocess_text(doc)
  clean_chunks = get_word_split(clean_doc)

  
  embed_doc = torch.tensor(np.zeros((1, 768)))
  embed_doc = embed_doc.to('cuda')

  for chunkNo, chunk in enumerate(clean_chunks):
    embed_chunk = getBERT(chunk)
    embed_doc = (embed_doc*chunkNo + embed_chunk)/(chunkNo+1)


  embedding[id-1] = (count[id-1]*embedding[id-1] + embed_doc)/(count[id-1]+1)
  count[id-1] = count[id-1]+1

  if i%5000==0:
    # save the files batchwise
    num = int(i/5000)
    embedding_tosave = embedding.cpu().detach().numpy()
    embedding_sparse = sp.csr_matrix(embedding_tosave)
    sp.save_npz("/content/drive/MyDrive/text gcn/node_feature/ironmarch/notifs_embedding/chunk/v{}".format(i+1), embedding_sparse)
    count_tosave = count.cpu().detach().numpy()
    count_sparse = sp.csr_matrix(count_tosave)
    sp.save_npz("/content/drive/MyDrive/text gcn/node_feature/ironmarch/notifs_embedding/chunk/count_v{}".format(i+1), count_sparse)


In [ ]:
embedding_tosave = embedding.cpu().detach().numpy()
embedding_sparse = sp.csr_matrix(embedding_tosave)
    
sp.save_npz("/content/drive/MyDrive/text gcn/node_feature/ironmarch/notifs_embedding/chunk/FINAL", embedding_sparse)